# 🎧 Moodle to MP3 Converter (Colab Edition)

**Steps:**
1. Export your Moodle cookies using a browser extension (see instructions below)
2. Upload `cookies.txt` when prompted
3. Paste your video links
4. Run all cells
5. Download your MP3 files!

---

### 🍪 How to Export Cookies

Use one of these browser extensions:
- **Chrome**: [Get cookies.txt LOCALLY](https://chrome.google.com/webstore/detail/get-cookiestxt-locally/cclelndahbckbenkjhflpdbgdldlbecc)
- **Firefox**: [cookies.txt](https://addons.mozilla.org/en-US/firefox/addon/cookies-txt/)

1. Log into your Moodle site
2. Click the extension icon → "Export" or "Download"
3. Save as `cookies.txt`

In [ ]:
# Step 2: Install/Update yt-dlp
!pip install -U yt-dlp -q
print("✅ yt-dlp ready!")

In [ ]:
# Step 1: Upload your cookies.txt file
from google.colab import files

print("📤 Upload your cookies.txt file (exported from your browser)")
uploaded = files.upload()

if 'cookies.txt' in uploaded:
    print("✅ cookies.txt uploaded successfully!")
else:
    # Rename if user uploaded with different name
    for filename in uploaded.keys():
        if filename.endswith('.txt'):
            with open('cookies.txt', 'wb') as f:
                f.write(uploaded[filename])
            print(f"✅ Saved {filename} as cookies.txt")
            break

In [ ]:
# Step 3: Paste your extracted links here (one per line)
links = """
https://example.cloudfront.net/video1.mp4
https://example.cloudfront.net/video2.mp4
"""

# Save to file
with open('links.txt', 'w') as f:
    f.write('\n'.join([l.strip() for l in links.strip().split('\n') if l.strip()]))
print(f"✅ Saved {len([l for l in links.strip().split(chr(10)) if l.strip()])} links")

In [ ]:
# Step 4: Download and convert to MP3 (using cookies for authentication)
import os

if os.path.exists('cookies.txt'):
    print("🍪 Using cookies.txt for authentication...")
    !yt-dlp --cookies cookies.txt -x --audio-format mp3 --audio-quality 192K -o "%(title)s.%(ext)s" -a links.txt
else:
    print("⚠️ No cookies.txt found - trying without authentication...")
    !yt-dlp -x --audio-format mp3 --audio-quality 192K -o "%(title)s.%(ext)s" -a links.txt

print("\n✅ Conversion complete!")

In [ ]:
# Step 4: Zip and download all MP3 files
import glob
from google.colab import files
import zipfile

mp3_files = glob.glob('*.mp3')
print(f"Found {len(mp3_files)} MP3 files")

if mp3_files:
    with zipfile.ZipFile('moodle_audio.zip', 'w') as zipf:
        for mp3 in mp3_files:
            zipf.write(mp3)
    files.download('moodle_audio.zip')
    print("✅ Downloading zip...")
else:
    print("❌ No MP3 files found. Check the download step above.")